In [13]:
import joblib as joblib
from sklearn import datasets
from skimage.feature import hog
from sklearn import preprocessing
from sklearn.datasets import fetch_openml
import numpy as np
from sklearn.svm import SVC
# Load the dataset
dataset = datasets.fetch_openml('mnist_784')
features = dataset.data 
labels = dataset.target
# Extract the hog features
list_hog_ft = []
for feature in features:
    ft = hog(feature.reshape((28, 28)), orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False)
    list_hog_ft.append(ft)
hog_features = np.array(list_hog_ft, 'float64')
# Normalize the features
pp = preprocessing.StandardScaler().fit(hog_features)
hog_features = pp.transform(hog_features)
# Create an SVM classifier
clf = SVC(C=5,gamma=.05)

# Perform the training
clf.fit(hog_features, labels)


SVC(C=5, gamma=0.05)

In [14]:
clf.score(hog_features, labels)

0.9879571428571429

In [15]:
joblib.dump((clf, pp), "digits_cls1.pkl", compress=3)

['digits_cls1.pkl']

In [2]:
import cv2
import joblib as joblib
from skimage.feature import hog
import numpy as np
# Load the classifier
clf,pp = joblib.load("digits_cls1.pkl")
# Read the input image 
im = cv2.imread("photo_2.jpg")
# Convert to grayscale and apply Gaussian filtering
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)
# Threshold the image
ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)

# Find contours in the image
ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# Get rectangles contains each contour
rects = [cv2.boundingRect(ctr) for ctr in ctrs]
for rect in rects:
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
    # Make the rectangular region around the digit
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
    # Resize the image
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    roi = cv2.dilate(roi, (3, 3))
    # Calculate the HOG features
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False)
    roi_hog_fd = pp.transform(np.array([roi_hog_fd], 'float64'))
    # Pridict digit
    nbr = clf.predict(roi_hog_fd)
    cv2.putText(im, str(int(nbr[0])), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)
cv2.namedWindow("Resulting Image with Rectangular ROIs", cv2.WINDOW_NORMAL)
cv2.imshow("Resulting Image with Rectangular ROIs", im)
cv2.waitKey(0)
cv2.destroyAllWindows()
    